In [1]:
# Imports
import pandas as pd
import os
from PIL import Image
import glob


In [ ]:
# Convert .xlsx to .csv
def xlsx_to_csv(excel_file: str, sheet_name: str, output_file: str) -> None:
    df = pd.read_excel(excel_file, sheet_name=sheet_name)
    df.to_csv(output_file, index=False)
    print(f"Converted {excel_file} to {output_file}")


In [9]:
excel_file = "./decisions.xlsx"
csv_file = "./decisions.csv"
sheet_name = "Sheet1"


In [ ]:
xlsx_to_csv(excel_file, sheet_name, csv_file)


In [27]:
# Fill empty fields
df = pd.read_csv(csv_file)
df['in_hand_item'].fillna('no_item', inplace=True)
df['target_mob'].fillna('no_mob', inplace=True)
df["decision_hearts"].fillna("no_decision_for_hearts", inplace=True)
df["decision_light"].fillna("no_decision_for_light", inplace=True)
df["decision_mob"].fillna("no_decision_for_mob", inplace=True)

df = df.dropna(subset=['hearts', 'activity', "light_lvl", "decision_activity"])

df.to_csv(csv_file, index=False)


C:\Users\sqdar\AppData\Local\Temp\ipykernel_1564\2447675307.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['in_hand_item'].fillna('no_item', inplace=True)
C:\Users\sqdar\AppData\Local\Temp\ipykernel_1564\2447675307.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

In [ ]:
# Compressing images
def compress_image(image_path, output_path, max_size=(800, 450), quality=85):
    try:
        with Image.open(image_path) as img:
            if img.mode == 'RGBA':
                img = img.convert('RGB')
                
            img.thumbnail(max_size)
            img.save(output_path, "JPEG", quality=quality)
            print(f"Image compressed and saved: {output_path}")
    except Exception as e:
        print(f"Error compressing {image_path}: {e}")

def compress_images_in_folder(input_folder, output_folder, max_size=(800, 450), quality=85):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        input_path = os.path.join(input_folder, filename)
        if os.path.isfile(input_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            output_path = os.path.join(output_folder, filename)
            compress_image(input_path, output_path, max_size, quality)

input_folder = "./raw_images"
output_folder = "./compressed_images"

compress_images_in_folder(input_folder, output_folder)


In [ ]:
# Crop only hearts and toolbar fragments
def process_screenshots(input_folder, hearts_output_folder, in_hand_output_folder):
    os.makedirs(hearts_output_folder, exist_ok=True)
    os.makedirs(in_hand_output_folder, exist_ok=True)

    heart_region_coords = (215, 370, 400,  420)
    in_hand_region_coords = (200, 410, 600,  450)

    screenshot_files = glob.glob(os.path.join(input_folder, '*.png'))

    for screenshot_path in screenshot_files:
        image = Image.open(screenshot_path).convert('RGB')

        heart_region = image.crop(heart_region_coords)
        heart_filename = os.path.join(hearts_output_folder, os.path.basename(screenshot_path))
        heart_region.save(heart_filename)

        in_hand_region = image.crop(in_hand_region_coords)
        in_hand_filename = os.path.join(in_hand_output_folder, os.path.basename(screenshot_path))
        in_hand_region.save(in_hand_filename)

        print(f"Processed {os.path.basename(screenshot_path)}")

input_folder = "./compressed_images"
hearts_output_folder = "./hearts_regions"
in_hand_output_folder = "./in_hand_regions"
process_screenshots(input_folder, hearts_output_folder, in_hand_output_folder)


In [16]:
df

,screenshot_title,activity,hearts,light_lvl,in_hand_item,target_mob,decision_activity,decision_hearts,decision_light,decision_mob
0,archering (100).png,archery,11,high,bow,no_mob,give_resistance,give_regeneration_2,no_decision_for_light,no_decision_for_mob
1,archering (108).png,archery,3,high,bow,no_mob,give_resistance,give_regeneration_4,no_decision_for_light,no_decision_for_mob
2,archering (112).png,archery,3,high,bow,no_mob,give_resistance,give_regeneration_4,no_decision_for_light,no_decision_for_mob
3,archering (12).png,archery,20,high,bow,no_mob,give_resistance,no_decision_for_hearts,no_decision_for_light,no_decision_for_mob
4,archering (1546).png,archery,15,high,bow,other,give_resistance,give_regeneration_1,no_decision_for_light,no_decision_for_mob
...,...,...,...,...,...,...,...,...,...,...
6086,swimming (684).png,swimming,4,high,no_item,no_mob,give_water_breathing,give_regeneration_4,no_decision_for_light,no_decision_for_mob
6087,swimming (688).png,swimming,4,high,no_item,no_mob,give_water_breathing,give_regeneration_4,no_decision_for_light,no_decision_for_mob
6088,swimming (690).png,swimming,4,high,no_item,no_mob,give_water_breathing,give_regeneration_4,no_decision_for_light,no_decision_for_mob
6089,swimming (694).png,swimming,4,high,no_item,no_mob,give_water_breathing,give_regeneration_4,no_decision_for_light,no_decision_for_mob


In [15]:
df.nunique()

screenshot_title     5576
activity                6
hearts                 21
light_lvl               3
in_hand_item            8
target_mob              7
decision_activity       6
decision_hearts         5
decision_light          3
decision_mob            4
dtype: int64